In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers faiss-cpu gradio pypdf sentencepiece

In [4]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import transformers
import torch
import gradio
import textwrap

In [5]:
model='meta-llama/Llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [7]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f2951598400>, model_kwargs={'temperature': 0})

In [9]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device':'cpu'})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
!unzip breast_cancer_clinical_trial.csv.zip -d /content/

Archive:  breast_cancer_clinical_trial.csv.zip
  inflating: /content/breast_cancer_clinical_trial.csv  


In [10]:
loader = CSVLoader('/content/bcct_500.csv',encoding='utf-8',csv_args={'delimiter':','})

In [11]:
data = loader.load()

In [12]:
data

[Document(page_content=": 0\nUnnamed: 0: 0\nNCT Number: NCT03275311\nStudy Title: Evolution of the Therapeutic Care in Metastatic Breast Cancer From 2008\nStudy URL: https://www.clinicaltrials.gov/study/NCT03275311\nStudy Status: Recruiting\nPhases: \nInclusion Criteria: \nExclusion Criteria: \nStart Date: July 2014\nCompletion Date: December 2024\nFacility: Institut de Cancérologie de l'Ouest - Paul Papin\nCity: Angers\nState: \nZip: 49933\nCountry: France\nCondition 1: Metastatic Breast Cancer\nCondition 2: \nCondition 3: \nCondition 4: \nCondition 5: \nCondition 6: \nCondition 7: \nCondition 8: \nCondition 9: \nCondition 10: \nCondition 11: \nCondition 12: \nCondition 13: \nCondition 14: \nCondition 15: \nCondition 16: \nCondition 17: \nCondition 18: \nCondition 19: \nCondition 20: \nCondition 21: \nCondition 22: \nCondition 23: \nCondition 24: \nCondition 25: \nCondition 26: \nCondition 27: \nCondition 28: \nCondition 29: \nCondition 30: \nCondition 31: \nCondition 32: \nCondition 

In [ ]:
vectorstore = FAISS.from_documents(data, embeddings)
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', return_source_documents=True, retriever=vectorstore.as_retriever())